# Black Coffer Sentiment Analysis Assignment

#### Importing necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import glob
import re
import nltk

#### Reading the Excel File

In [2]:
df=pd.read_excel('Input.xlsx')

#### Having a glance at the top 5 rows of the dataset

In [3]:
df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


#### Finding the shape of the dataset

In [4]:
df.shape

(114, 2)

#### Adding all the headings in a text file named headings

In [5]:
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, 'lxml')
    article_text_list =[]
    element1=soup.find_all('h1',class_="entry-title")
    element2=soup.find_all('h1',class_="tdb-title-text")
    if element1:
        article_text_list.append(element1[0].text)
    elif element2:
        article_text_list.append(element2[0].text)
    else:
        article_text_list.append("Heading not found")
    with open(f"headings.txt", 'a') as file:
        for item in article_text_list:
            file.write(item + '\n')

#### From the above heading text file we have observed that two URLs with index 24 and 37 doesnt exist, hence we will drop them.

In [6]:
df=df.drop([24,37])

#### Resetting the index after dropping the Invalid URLs

In [7]:
df = df.reset_index(drop=True)

#### Web Scraping the title and content of the URLs using Beautiful soup in a text_files folder, with name of each file based on a URL_ID 

In [8]:
for index, row in df.iterrows():
    article_text_list =[]
    url = row['URL']
    url_id = row['URL_ID']
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, 'lxml')
    
    element1=soup.find_all('h1',class_="entry-title")
    element2=soup.find_all('h1',class_="tdb-title-text")
    if element1:
        article_text_list.append(element1[0].text)
    elif element2:
        article_text_list.append(element2[0].text)
    else:
        article_text_list.append("Heading not found")
    
    div_element1 = soup.find('div', class_='td-post-content tagdiv-type')
    div_element2 = soup.find('div', class_='tdb-block-inner td-fix-index') 
    div_element0 = soup.find_all('div', class_='td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')

    if div_element1:
        p_elements = div_element1.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li'], class_=None)

        for p_element in p_elements:
            article_text_list.append(p_element.get_text())
        with open(f"text_files/{url_id}.txt", 'a', encoding='utf-8') as file:
            for item in article_text_list:
                file.write(f"{item}\n")
                
    elif div_element0:
        div_element0=div_element0[0].text
        div_element0=soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6','li'],class_=None)

        for div_elements in div_element0:
            article_text_list.append(div_elements.text)
        with open(f"text_files/{url_id}.txt", 'a', encoding='utf-8') as file:
            for item in article_text_list:
                file.write(f"{item}\n")
    
    elif div_element2:
        p_elements = div_element2.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li'], class_=None)

        for p_element in p_elements:
            article_text_list.append(p_element.get_text())  
        with open(f"text_files/{url_id}.txt", 'a', encoding='utf-8') as file:
            for item in article_text_list:
                file.write(f"{item}\n")
                
    else:
        article_text_final.append("NOTTTTTTTTTTTTTTTTTTTTT FOUNDDDDDDDDDDDDDDDDDDDDDDDDDDDD")
        with open(f"text_files/{url_id}.txt", 'a', encoding='utf-8') as file:
            for item in article_text_list:
                file.write(f"{item}\n")
            

#### Web scraping a URL seperately which has a different sub div class compared to others

In [9]:
url='https://insights.blackcoffer.com/rise-of-electric-vehicle-and-its-impact-on-livelihood-by-the-year-2040/'
url_id=7050.0
webpage = requests.get(url).text
soup = BeautifulSoup(webpage, 'lxml')
article_text_list =[]
div_element = soup.find('div', class_='td-post-content tagdiv-type')
p_elements = div_element.find_all('p', class_='has-text-align-left')

for p_element in p_elements:
    article_text_list.append(p_element.get_text())
with open(f"text_files/{url_id}.txt", 'a', encoding='utf-8') as file:
    for item in article_text_list:
        file.write(f"{item}\n")

#### Checking for empty text files, to make sure the data is scraped from each URL

In [10]:
import os

folder_path = "text_files"

file_list = os.listdir(folder_path)

text_files = [file for file in file_list if file.endswith(".txt")]

empty_files = []

for file_name in text_files:
    file_path = os.path.join(folder_path, file_name)
    if os.path.getsize(file_path) == 0:
        empty_files.append(file_name)

if empty_files:
    print("Empty text files:")
    for file_name in empty_files:
        print(file_name)
else:
    print("No empty text files found.")

No empty text files found.


#### Reading the first file and doing analysis

In [11]:
file_path = "text_files/123.0.txt"  
try:
    lines_with_fullstops = []  

    with open(file_path, 'r') as file:
        for line in file:
            line = line.rstrip()          
            if not line.endswith('.'):
                line += '.'              
            lines_with_fullstops.append(line)
        
except FileNotFoundError:
    print(f"The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

#### Calculating the number of files in a folder text_files

In [12]:
import os
folder_path = 'text_files'
items = os.listdir(folder_path)
files = [item for item in items if os.path.isfile(os.path.join(folder_path, item))]
number_of_files = len(files)
print(f"Number of files in the folder: {number_of_files}")

Number of files in the folder: 112


#### Extracting the data from each text file and storing it into a variable 'sentences' w.r.t to its URL_ID. Breakdown of paragraphs into individual sentences and storing them as values in a list. 

In [13]:
df['sentences']=None
folder_path = 'text_files'

file_list = os.listdir(folder_path)

for filename in file_list:
    lines_with_fullstops = []
    parts = filename.split('.')
    url_id_str = '.'.join(parts[:2])
    url_id=float(url_id_str)
    try:
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            for line in file:
                line = line.rstrip()
                if not line.endswith('.'):
                    line += '.'  
                lines_with_fullstops.append(line)
         
            sentences = []
            for paragraph in lines_with_fullstops:
                paragraph_sentences = paragraph.split('.')
                paragraph_sentences = [sentence.strip() for sentence in paragraph_sentences if sentence.strip()]
                sentences.extend(paragraph_sentences)

    except FileNotFoundError:
        print(f"The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    
    matching_indices = df[df['URL_ID'] == url_id].index.tolist()
    if matching_indices:
        for index in matching_indices:
            df.at[index, 'sentences'] = sentences
    else:
        print(f"No match found for '{url_id}' in 'URL_ID'")         

In [14]:
len(df['sentences'])

112

In [15]:
df.head(3)

,URL_ID,URL,sentences
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,[Rise of telemedicine and its Impact on Liveli...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,[Rise of e-health and its impact on humans by ...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,[Rise of e-health and its impact on humans by ...


Extracting stop words from different stop word text files and after lowering into smallcase storing it into a list

In [16]:
folder_path = "StopWords"
stop_words = []

def extract_first_word(line):
    words = line.split()
    if words:
        return words[0].strip()  
    else:
        return None  

file_paths = glob.glob(os.path.join(folder_path, "*.txt"))

for file_path in file_paths:
    try:
        with open(file_path, 'r') as file:
            for line in file:
                first_word = extract_first_word(line)
                if first_word:
                    first_word=first_word.lower()
                    stop_words.append(first_word)
    except FileNotFoundError:
        print(f"The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred while processing '{file_path}': {str(e)}")
    

#### Cleaning the data:
- Converting into lowercase
- Removing stop words and special characters
- Storing the cleaned data into a variable named stopwords_removed

In [17]:
from nltk.tokenize import word_tokenize

def clean_sentence(sentence, stop_words):
    sentence = sentence.lower()    
    sentence = re.sub(r'[^A-Za-z0-9 ]', ' ', sentence)    
    words = word_tokenize(sentence)    
    filtered_words = [word for word in words if word not in stop_words]    
    cleaned_sentence = ' '.join(filtered_words) 
    return cleaned_sentence

In [18]:
df['stopwords_removed']=None
for index,i in enumerate(df['sentences']):
    cleaned_sentences = []
    for sentence in i:
        cleaned_sentences.append(clean_sentence(sentence, stop_words))
    df.at[index,'stopwords_removed'] = cleaned_sentences
        

#### Calculating the Total words in a URL

In [19]:
df['total_word_count']=None
for index,i in enumerate(df['stopwords_removed']):
    total_word_count = 0
    for sentence in i:
        words = word_tokenize(sentence)
        total_word_count += len(words)
    df.at[index,'total_word_count'] = total_word_count

In [20]:
from nltk.tokenize import word_tokenize

cleaned_sentences = []
def clean_sentence(sentence, stop_words):
    sentence = sentence.lower()    
    sentence = re.sub(r'[^A-Za-z0-9 ]', ' ', sentence)    
    words = word_tokenize(sentence)    
    filtered_words = [word for word in words if word not in stop_words]    
    cleaned_sentence = ' '.join(filtered_words) 
    return cleaned_sentence

for sentence in sentences:
    cleaned_sentences.append(clean_sentence(sentence, stop_words))

#### Extracting Positive words and Negative words from the text files

In [21]:
file_path='MasterDictionary/positive-words.txt'
positive_words=[]
with open(file_path, 'r') as file:
    for line in file:
        first_word = extract_first_word(line)
        if first_word:
            if first_word not in stop_words:
                positive_words.append(first_word)

In [22]:
file_path='MasterDictionary/negative-words.txt'
negative_words=[]
with open(file_path, 'r') as file:
    for line in file:
        first_word = extract_first_word(line)
        if first_word:
            if first_word not in stop_words:
                negative_words.append(first_word)

#### Calculating positive and negative word count for each URL and storing into variables

In [23]:
df['positive_score']=None
for index,i in enumerate(df['stopwords_removed']):
    positive_score = 0
    for sentence in i:
        words = word_tokenize(sentence)
        for word in words:
            if word.lower() in positive_words:
                positive_score += 1
    df.at[index,'positive_score'] = positive_score
    

df['negative_score']=None
for index,i in enumerate(df['stopwords_removed']):
    negative_score = 0
    for sentence in i:
        words = word_tokenize(sentence)
        for word in words:
            if word.lower() in negative_words:
                negative_score += 1
    df.at[index,'negative_score'] = negative_score

#### Calculating the number of sentences in each article

In [24]:
df['no_of_sentences']=None
for index,i in enumerate(df['stopwords_removed']):
    no_of_sentences= 0
    no_of_sentences = len(i)
    df.at[index,'no_of_sentences'] = no_of_sentences

#### Function to count syllables in a word

In [25]:
def count_syllables(word):
    if word.endswith("es") or word.endswith("ed"):
        return 1     
    vowels = "AEIOUaeiou"
    count = 0
    prev_char = ''
    
    for char in word:
        if char in vowels and (prev_char != 'y' or prev_char == word[0]):
            count += 1
        prev_char = char
    return count

#### Calculating Complex word count for each URL

In [26]:
df['complex_word_count']=None
for index,i in enumerate(df['stopwords_removed']):
    complex_word_count = 0
    for sentence in i:
        words = word_tokenize(sentence)
        for word in words:
            if count_syllables(word) > 2:
                complex_word_count += 1
    df.at[index,'complex_word_count'] = complex_word_count

In [27]:
df.head(2)

,URL_ID,URL,sentences,stopwords_removed,total_word_count,positive_score,negative_score,no_of_sentences,complex_word_count
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,[Rise of telemedicine and its Impact on Liveli...,"[rise telemedicine impact livelihood 2040, tel...",822,79,23,90,490
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,[Rise of e-health and its impact on humans by ...,"[rise health impact humans 2030, rise health e...",274,38,13,25,151


#### Calculating Polarity Score
- This is the score that determines if a given text is positive or negative in nature. 
- It is calculated by using the formula: Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)


In [28]:
df['Polarity_score'] = (df['positive_score'] - df['negative_score'])/ ((df['positive_score'] + df['negative_score']) + 0.000001)

#### Calculating Subjectivity Score
- Subjectivity Score: This is the score that determines if a given text is objective or subjective. 
- It is calculated by using the formula: Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)


In [29]:
df['Subjectivity_score'] = (df['positive_score'] + df['negative_score'])/ ((df['total_word_count']) + 0.000001)

#### Calculating Average sentence Length
- Average Sentence Length = the number of words / the number of sentences

In [30]:
df['Average_sentence_length']=df['total_word_count']/df['no_of_sentences']

#### Percentage of Complex words
- Percentage of Complex words = the number of complex words / the number of words 

In [31]:
df['Percentage_of_complex_words']=df['complex_word_count']/df['total_word_count']

#### Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

In [32]:
df['Fog Index']=0.4 * (df['Average_sentence_length'] + df['Percentage_of_complex_words'])

#### Average Number of Words Per Sentence
- The formula for calculating is: Average Number of Words Per Sentence = the total number of words / the total number of sentences


In [33]:
df['Average_no_of words_per_sentence']=df['total_word_count']/df['no_of_sentences']

#### Syllable Count Per Word
- We count the number of Syllables in each word of the text by counting the vowels present in each word. We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable

In [34]:
df['syllable per word']=None
for index,i in enumerate(df['stopwords_removed']):
    syllable_dict = {}
    for sentence in i:
        words = word_tokenize(sentence)
        for word in words:
            syllable_dict[word] = count_syllables(word)
    df.at[index,'syllable per word'] = syllable_dict

#### Calculating Personal Pronouns
- To calculate Personal Pronouns mentioned in the text, we use regex to find the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken so that the country name US is not included in the list.


In [35]:
df['personal_pronoun_count']=None
for index,i in enumerate(df['sentences']):
    pronoun_pattern = r'\b(?:I|we|my|ours|us)\b'
    personal_pronoun_count = 0
    for sentence in i:
        matches = re.findall(pronoun_pattern, sentence, re.IGNORECASE)
        matches = [match for match in matches if match.lower() != "us"]
        personal_pronoun_count += len(matches)
    df.at[index,'personal_pronoun_count'] = personal_pronoun_count

#### Calculating total characters in each URL

In [36]:
df['total_characters']=None
for index,i in enumerate(df['stopwords_removed']):
    total_characters = 0
    for sentence in i:
        words = word_tokenize(sentence)
        for word in words:
            total_characters += len(word)
    df.at[index,'total_characters'] = total_characters

#### Average Word Length is calculated by the formula:
- Average Word Length = Sum of the total number of characters in each word / Total number of words

In [37]:
df['average_word_length'] = df['total_characters'] / df['total_word_count']

In [38]:
df.drop(['sentences','stopwords_removed','no_of_sentences','total_characters'],axis=1,inplace=True)

In [39]:
df.head(5)

,URL_ID,URL,total_word_count,positive_score,negative_score,complex_word_count,Polarity_score,Subjectivity_score,Average_sentence_length,Percentage_of_complex_words,Fog Index,Average_no_of words_per_sentence,syllable per word,personal_pronoun_count,average_word_length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,822,79,23,490,0.54902,0.124088,9.133333,0.596107,3.891776,9.133333,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",2,7.833333
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,274,38,13,151,0.490196,0.186131,10.96,0.551095,4.604438,10.96,"{'rise': 2, 'health': 2, 'impact': 2, 'humans'...",3,8.109489
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,548,21,27,241,-0.125,0.087591,5.956522,0.439781,2.558521,5.956522,"{'rise': 2, 'health': 2, 'impact': 2, 'humans'...",4,7.282847
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,633,34,26,297,0.133333,0.094787,8.915493,0.469194,3.753875,8.915493,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",7,7.309637
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,633,34,26,297,0.133333,0.094787,8.915493,0.469194,3.753875,8.915493,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",7,7.309637


In [40]:
len(df.columns)

15

In [41]:
df=df.rename(columns={'positive_score' : 'POSITIVE SCORE' , 'negative_score': 'NEGATIVE SCORE', 'total_word_count': 'WORD COUNT', 'complex_word_count': 'COMPLEX WORD COUNT', 'Polarity_score': 'POLARITY SCORE', 'Subjectivity_score': 'SUBJECTIVITY SCORE', 'Average_sentence_length': 'AVG SENTENCE LENGTH', 'Percentage_of_complex_words': 'PERCENTAGE OF COMPLEX WORDS', 'Fog Index': 'FOG INDEX', 'Average_no_of words_per_sentence': 'AVG NUMBER OF WORDS PER SENTENCE', 'syllable per word': 'SYLLABLE PER WORD', 'personal_pronoun_count': 'PERSONAL PRONOUNS', 'average_word_length': 'AVG WORD LENGTH'})

In [42]:
df.head(2)

,URL_ID,URL,WORD COUNT,POSITIVE SCORE,NEGATIVE SCORE,COMPLEX WORD COUNT,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,822,79,23,490,0.54902,0.124088,9.133333,0.596107,3.891776,9.133333,"{'rise': 2, 'telemedicine': 6, 'impact': 2, 'l...",2,7.833333
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,274,38,13,151,0.490196,0.186131,10.96,0.551095,4.604438,10.96,"{'rise': 2, 'health': 2, 'impact': 2, 'humans'...",3,8.109489


In [43]:
df.to_excel('final_output.xlsx', index=False)

# Finally, The END !!!!!!